## 1. Construct the emission matrix p(w/t)
In the trigram model, I label the word using the tag consists of its own tag with the previous tag.  

In [3]:
## Construct bag of word for word and tag seperately
word_count = {}
tag_count = {}
with open ("C:/Users/Xin/Desktop/Github-repo/Natural_Language_Processing_Projects/Project_3_Sequence_Tagging/hw3-data/train.txt") as train_file:
    for line in train_file:
        line_list = line.split()
        temp_tag_list = []
        for i in range(len(line_list)):
            temp = line_list[i].split('/')
            if temp[0] not in word_count:
                word_count[temp[0]]=1
            if temp[0] in word_count:
                word_count[temp[0]]+=1
            temp_tag_list.append(temp[1])
        ## Construct bigram tags
        temp_tag_list_prev = ["<s>"]
        for i in range(len(temp_tag_list)-1):
            temp_tag_list_prev.append(temp_tag_list[i])
        for i in range(len(temp_tag_list)):
            tag_temp= "".join([temp_tag_list_prev[i],temp_tag_list[i]])
            if tag_temp not in tag_count:
                tag_count[tag_temp] = 1
            else:
                tag_count[tag_temp] += 1
### word_count
#{'azaleas': 5,
# 'termination': 3,
# 'planting': 26,
# ...

### tag_count
#{'<s>A': 28,
# '<s>D': 11005,
# '<s>E': 1978,
# '<s>F': 12612,
# '<s>N': 153233,
# '<s>R': 11318,
# 'AA': 2564,
# 'AD': 19,
# 'AE': 2,
# ... 

In [10]:
## Construct the data structure to store the c(t,w). Here t is the double tag created above.  
# Use the dictionary to store the counts
# For the dictionary word_tag_pair_counts, the keys are the tags in the training set, the value is a dictionary, in which the keys 
# are the words, the the value are the counts for this word-key pair 
tag_word_pair_counts = {}

with open ("C:/Users/Xin/Desktop/Github-repo/Natural_Language_Processing_Projects/Project_3_Sequence_Tagging/hw3-data/train.txt") as train_file:
    for line in train_file:
        line_list = line.split()
        
        temp_tag_list_uni = []
        temp_tag_list_double = []
        temp_word_list = []
        for i in range(len(line_list)):
            temp = line_list[i].split('/')
            temp_word_list.append(temp[0])
            temp_tag_list_uni.append(temp[1])
        
        ## Construct the double tag    
        temp_tag_list_uni_prev = ["<s>"]
        for i in range(len(temp_tag_list_uni)-1):
            temp_tag_list_uni_prev.append(temp_tag_list_uni[i])
        for i in range(len(temp_tag_list_uni)):
            temp_tag_list_double.append("".join([temp_tag_list_uni_prev[i],temp_tag_list_uni[i]]))
        
        for i in range(len(temp_tag_list_double)):
            if temp_tag_list_double[i] not in tag_word_pair_counts:
                tag_word_pair_counts[temp_tag_list_double[i]] = {}
                tag_word_pair_counts[temp_tag_list_double[i]][temp_word_list[i]] = 1
            else:
                if temp_word_list[i] not in tag_word_pair_counts[temp_tag_list_double[i]]:
                    tag_word_pair_counts[temp_tag_list_double[i]][temp_word_list[i]] = 1
                else:
                    tag_word_pair_counts[temp_tag_list_double[i]][temp_word_list[i]] += 1
#tag_word_pair_counts
#{'<s>A': {'--': 8,
#  'and': 1,
#  'basketball': 1,
#  'but': 1,
#  'criminal': 1,
#  'frankly': 1,
#  'i': 2,
#  'like': 1,
#  'not': 3,
# ...
# ...

In [14]:
## Construct the emission matrix, each entry is p(w/t), I use add delta smoothing to deal with unknown word

## I use a dictionary of dictionary to represent this matrix
## The key of the outer dictionary is the double tag, and the the key of the sub-dictionary is their corresponding word
emission_matrix = {}
delta = 0.1 
V = len(word_count) + 1 ## assum only one kind of unseen words
for tag in tag_count:
    emission_matrix[tag]={}
    sum_tag_allword = 0
    for word in tag_word_pair_counts[tag]:
        sum_tag_allword += tag_word_pair_counts[tag][word]
    sum_tag_allword += V*delta ## Add delta smoothing
    for word in tag_word_pair_counts[tag]:
        emission_matrix[tag][word] = (tag_word_pair_counts[tag][word]+delta)/sum_tag_allword
    emission_matrix[tag]['unk'] = delta/sum_tag_allword
#emission_matrix
#{'<s>A': {'--': 0.0039187227866473145,
#  'and': 0.0005321722302854379,
#  'basketball': 0.0005321722302854379,
#  'but': 0.0005321722302854379,
#  'criminal': 0.0005321722302854379,
#  'frankly': 0.0005321722302854379,
# ...

In [26]:
## Obtain p(you/t)
count = 0
for tag in tag_count:    
    if "you" not in emission_matrix[tag]:
        continue
    else:
        count += 1 
        print ("p(you/{}): {}".format(tag,emission_matrix[tag]['you']))
print (count)

p(you/ND): 0.6603412430998639
p(you/FN): 0.011478710196960038
p(you/NN): 0.012527265507569068
p(you/NE): 0.0018945022288261513
p(you/<s>E): 0.0002738361961662933
p(you/<s>R): 0.01498090888672606
p(you/AD): 0.005393586005830904
p(you/AA): 0.0037149684988051273
p(you/FA): 0.0005169172932330828
p(you/<s>D): 0.15440815700705304
p(you/FD): 0.27144626052501236
p(you/<s>F): 7.50801993038018e-05
p(you/RE): 0.0004437273093989512
p(you/RD): 0.3818807884113037
p(you/<s>N): 0.012243675614405687
p(you/RR): 0.009069113375443773
p(you/NR): 0.015876539073624258
p(you/NA): 0.002756439222774514
p(you/DD): 0.008167176959649516
p(you/RA): 0.0018850574712643677
p(you/DR): 0.017379339542760372
p(you/ED): 0.05048143053645117
p(you/DN): 0.026242724930827212
p(you/FE): 0.0005183788878416588
p(you/EN): 0.02583201267828843
p(you/ER): 0.007919621749408984
p(you/RN): 0.017230509955683166
p(you/FR): 0.010583103764921947
p(you/AN): 0.0013225255972696246
29
